In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

# 读取数据集
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 处理 SalePrice 
prices = pd.DataFrame({'price':train['SalePrice'], 'log(price+1)':np.log1p(train['SalePrice'])})
# prices.hist()
train['SalePrice'] = np.log1p(train['SalePrice'])

# GrLivArea 
train['GrLivArea'] = train['GrLivArea'][train['GrLivArea']<4500]

# 合并训练集和测试机
all_data = pd.concat((train.loc[:,"MSSubClass":"SaleCondition"],
                     test.loc[:, "MSSubClass":"SaleCondition"]))

# 处理 偏斜>0.5
numeric_feats = all_data.dtypes[all_data.dtypes!='object'].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))
skewed_feats = skewed_feats[skewed_feats > 0.5]
skewed_feats = skewed_feats.index
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

# 转换成虚拟变量
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(all_data.mean())

X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train["SalePrice"]

In [62]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax


# 读取数据集
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_ID = train['Id']; test_ID = test['Id']
train.drop(['Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)

train = train[train.GrLivArea < 4500]
train.reset_index(drop=True, inplace=True)

train['SalePrice'] = np.log1p(train['SalePrice'])
y = train.SalePrice.reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

features = pd.concat([train_features, test_features]).reset_index(drop=True)
print(features.shape)
features['MSSubClass'] = features['MSSubClass'].apply(str)
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

features['Functional'] = features['Functional'].fillna('Typ')
features['Electrical'] = features['Electrical'].fillna('SBrkr')
features['KitchenQual'] = features['KitchenQual'].fillna('TA')
features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0])
features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])
features['PoolQC'] = features['PoolQC'].fillna('None')

for feat in ['GarageYrBlt', 'GarageArea', 'GarageCars']:
    features[feat] = features[feat].fillna(0)
for feat in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    features[feat].fillna('None')
for feat in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
    features[feat].fillna('None')
features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

objects = features.dtypes[features.dtypes=='object'].index
features.update(features[objects].fillna('None'))
features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

numerics = features.dtypes[features.dtypes!='object'].index
skew_features = features[numerics].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index
for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))


    
features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']
features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']

features['Total_sqr_footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                 features['1stFlrSF'] + features['2ndFlrSF'])

features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                               features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))

features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
                              features['EnclosedPorch'] + features['ScreenPorch'] +
                              features['WoodDeckSF'])

# simplified features
features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
features['has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

    

other_feats = X[X.isnull().sum()[X.isnull().sum() != 0].index]
for feat in other_feats:
    features[feat] = features[feat].fillna(0)
    
X = pd.get_dummies(features)
X_train = X[:len(y)]
X_test = X[len(y):]

overfit = []
for feat in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 99.94:
        overfit.append(i)

X_train = X_train.drop(overfit, axis=1).copy()
X_test = X_test.drop(overfit, axis=1).copy()

(2917, 79)


## Models

In [63]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNetCV, LassoCV, LassoLarsCV
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline 

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


kfolds = KFold(n_splits=10, shuffle=True, random_state=42)


def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y, scoring='neg_mean_squared_error', cv=5))
    return rmse


"""Ridge"""
alpha_ridge = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alpha_ridge, cv=kfolds))

"""Lasso"""
alpha_lasso = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e6,
                                               alphas=alpha_lasso,
                                               random_state=42,
                                               cv=kfolds))
"""Elasticnet"""
alpha_ela = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
ela = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e6,
                                                   alphas=alpha_ela,
                                                   cv=kfolds,
                                                   random_state=42,
                                                    l1_ratio = e_l1ratio
                                                   ))
"""SVR"""
svr = make_pipeline(RobustScaler(), SVR(C=20, epsilon=0.008, gamma=0.0003))

"""xgboost"""
xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                      max_depth=3, min_child_weight=0,
                      gamma = 0, subsample=0.7,
                      colsample_bytree=0.7,
                      objective='reg:linear', nthread=-1,
                      scale_pos_weight=1, seed=27,
                      reg_alpha=0.00006, random_state=42)

"""gbr"""
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                          max_depth=4, max_features='sqrt',
                          min_samples_leaf=15, min_samples_split=10,
                          loss='huber', random_state=42)

"""lightgbm"""
lightgbm = LGBMRegressor(objective='regression',
                         num_leaves=4,
                         learning_rate=0.01,
                         n_estimators=5000,
                         max_bin=200,
                         bagging_fraction=0.75,
                         bagging_freq=5,
                         bagging_seed=7,
                         feature_fraction=0.2,
                         feature_fraction_seed=7,
                         verbose=-1)


score = rmse_cv(ridge)
print('Ridge: ', score.mean(), score.std())
score = rmse_cv(lasso)
print('Lasso: ', score.mean(), score.std())
score = rmse_cv(ela)
print('ela: ', score.mean(), score.std())
score = rmse_cv(svr)
print('svr: ', score.mean(), score.std())
score = rmse_cv(xgboost)
print('xgboost: ', score.mean(), score.std())
score = rmse_cv(gbr)
print('gbr: ', score.mean(), score.std())
score = rmse_cv(lightgbm)
print('lightgbm: ', score.mean(), score.std())


ridge = ridge.fit(X_train, y)
lasso = lasso.fit(X_train, y)
ela = ela.fit(X_train, y)
xgboost = xgboost.fit(X_train, y)
gbr = gbr.fit(X_train, y)
lightgbm = lightgbm.fit(X_train, y)


pred = 0.3*lasso.predict(X_test) + \
        0.1*ridge.predict(X_test) + \
        0.1*ela.predict(X_test) + \
        0.3*xgboost.predict(X_test) + \
        0.1*gbr.predict(X_test) + \
        0.1*lightgbm.predict(X_test)

Ridge:  0.11168469485302249 0.006239395881046382
Lasso:  0.10852487811653169 0.006505856972357174
ela:  0.10852686521478012 0.006469188908459015
svr:  0.13588143757711868 0.009461465491795013
[22:48:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\Voyager\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Voyager\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:49:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\Voyager\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:49:45] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[22:50:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\Voyager\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:50:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgboost:  0.11205245075036237 0.006208896979159475
gbr:  0.1151789248419383 0.007841011860906279
lightgbm:  0.11575148572286598 0.005497098894733727


C:\Users\Voyager\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:53:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [64]:
res = pd.DataFrame({'id':test_ID, 'SalePrice': np.expm1(pred)})
res.to_csv('submission_2019_12_19.csv', index=False)